# Hybrid Search Test

## openai & prompt 설정

In [ ]:
# OpenAI API 설정 및 라이브러리 import
from openai import OpenAI
import json
import os
from typing import List, Dict
import dotenv
import sys
import os

# 프로젝트 루트 디렉토리를 Python 경로에 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


dotenv.load_dotenv()

In [ ]:
# openai 설정

# API 키 설정 (환경변수에서 가져오기)
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")  # 환경변수에 API 키를 설정해주세요
)


response_format = {
    "type": "json_object",
    "json_schema": {
            "name": "translate_result",
            "schema": {
                "type": "object",
                "properties": {
                    "text": {"type": "string"},
                    "translated": {"type": "string"},
                    "keyword": {"type": "array", "items": {"type": "string"}}
                },
                "required": ["text", "translated", "keyword"],
                "additionalProperties": False
            }
        }
}


def chat(
    messages: List[Dict[str, str]], 
    model: str = "gpt-4o-mini",
    response_format: dict = None,
    **kwargs
) -> str:
    """
    OpenAI GPT-4o-mini API를 사용하여 채팅 완성을 수행합니다.
    
    Args:
        messages: 대화 메시지 리스트 [{"role": "user", "content": "메시지"}]
        model: 사용할 모델명 (기본값: gpt-4o-mini)
        **kwargs: OpenAI API 매개변수들
            - temperature: 창의성 조절 (0.0-2.0, 기본값: 0.7)
            - max_tokens: 최대 토큰 수 (기본값: 1000)
            - top_p: 확률 임계값 (기본값: 0.95)
            - frequency_penalty: 빈도 페널티 (기본값: 0.0)
            - presence_penalty: 존재 페널티 (기본값: 0.0)
            - stream: 스트리밍 여부 (기본값: False)
            - 기타 OpenAI API가 지원하는 모든 매개변수
        
    Returns:
        GPT 응답 텍스트
    """
    # 기본값 설정
    default_params = {
        "temperature": 0.7,
        "max_tokens": 1000,
        "top_p": 0.95,
        "frequency_penalty": 0.0,
        "presence_penalty": 0.0
    }
    
    # 기본값과 사용자 입력 병합
    params = {**default_params, **kwargs}
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            response_format=response_format,
            **params
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        print(f"API 호출 중 오류 발생: {e}")
        return None


In [ ]:
# MongoDB 연결 테스트
import json
import os
import pymongo
from pymongo import MongoClient

print("=" * 50)
print("MongoDB 연결 테스트 시작")
print("=" * 50)

# 설정 파일 로드
with open('../configs/config.json', 'r', encoding='utf-8') as f:
    config = json.load(f)

# MongoDB 클라이언트 연결
mongodb_client = MongoClient(os.getenv("MONGODB_URI"))
print("🔗 MongoDB 클라이언트 연결 완료")

try:
    # 연결 상태 확인
    mongodb_client.admin.command('ping')
    print("✅ MongoDB 연결 성공!")
    
    # 서버 정보 가져오기
    server_info = mongodb_client.server_info()
    print(f"📊 MongoDB 버전: {server_info['version']}")
    
    # 데이터베이스 목록 확인
    db_list = mongodb_client.list_database_names()
    print(f"📁 사용 가능한 데이터베이스: {db_list}")
    
    # 현재 데이터베이스 정보
    current_db = mongodb_client[config['path']['db_name']]
    print(f"🎯 현재 데이터베이스: {config['path']['db_name']}")
    
    # 컬렉션 목록 확인
    collections = current_db.list_collection_names()
    print(f"📋 컬렉션 목록: {collections}")
    
    # 타겟 컬렉션 확인
    target_collection = current_db[config['path']['collection_name']]
    print(f"🎯 타겟 컬렉션: {config['path']['collection_name']}")
    
    # 컬렉션 통계 정보
    stats = current_db.command("collStats", config['path']['collection_name'])
    print(f"📈 문서 개수: {stats['count']:,}")
    print(f"💾 컬렉션 크기: {stats['size']:,} bytes ({stats['size']/1024/1024:.2f} MB)")
    
    # 샘플 문서 확인
    sample_doc = target_collection.find_one()
    if sample_doc:
        print(f"📄 샘플 문서 키: {list(sample_doc.keys())}")
        print(f"📄 샘플 문서 ID: {sample_doc.get('_id', 'N/A')}")
    else:
        print("⚠️ 컬렉션에 문서가 없습니다.")
    
    # 인덱스 정보 확인
    indexes = target_collection.list_indexes()
    print(f"🔍 인덱스 정보:")
    for idx in indexes:
        print(f"   - {idx['name']}: {idx['key']}")
    
    print("=" * 50)
    print("✅ MongoDB 연결 테스트 완료!")
    print("=" * 50)
    
except pymongo.errors.ConnectionFailure as e:
    print(f"❌ MongoDB 연결 실패: {e}")
except pymongo.errors.ServerSelectionTimeoutError as e:
    print(f"❌ 서버 선택 타임아웃: {e}")
except Exception as e:
    print(f"❌ 예상치 못한 오류: {e}")


In [ ]:
from prompts.prompts import load_prompt

translate_prompt = load_prompt("translate")
print(translate_prompt)

In [ ]:
# messages = [
#     {"role": "system", "content": translate_prompt},
#     {"role": "user", "content": "안녕하세요! 파이썬에 대해 간단히 설명해주세요."}
# ]

# response = chat(messages)

## 데이터 불러오기

In [ ]:
import pandas as pd

query_df = pd.read_csv("../data/helloworld_test_query.csv")

In [ ]:
# CSV 파일 불러오기 및 데이터프레임 생성
import pandas as pd

# CSV 파일 불러오기
query_df = pd.read_csv('../data/helloworld_test_query.csv')
print(f"불러온 데이터 개수: {len(query_df)}")
print("\n데이터프레임 컬럼:")
print(query_df.columns.tolist())
print("\n첫 3개 행:")
print(query_df.head(3))


In [ ]:
# 번역 프롬프트 가져오기
from prompts.prompts import load_prompt

translate_prompt = load_prompt("translate")
print("번역 프롬프트 로드 완료")

# 번역 함수 정의
def translate_query(query_text):
    """번역 쿼리를 GPT-4o-mini로 번역하여 JSON 응답을 받는다"""
    messages = [
        {"role": "system", "content": translate_prompt},
        {"role": "user", "content": query_text}
    ]
    
    try:
        response = chat(messages, temperature=0.1)
        result = json.loads(response)
        return result
    except Exception as e:
        print(f"번역 오류: {e}")
        return None


## 번역 & 키워드 추출

In [ ]:
# 모든 쿼리 처리
import json
from tqdm import tqdm
import time

# 결과를 저장할 리스트
all_results = []
translated_results = []
keyword_results = []

print(f"총 {len(query_df)}개의 쿼리를 처리합니다...")

# 각 쿼리 처리
for idx, row in tqdm(query_df.iterrows(), total=len(query_df), desc="번역 진행"):
    query_text = row['translated_query']
    
    # 번역 수행
    result = translate_query(query_text)
    
    if result is not None:
        # 전체 결과 저장
        all_results.append(result)
        
        # 개별 필드 저장
        translated_results.append(result.get('translated', ''))
        keyword_results.append(result.get('keyword', []))
        
        print(f"[{idx+1}/{len(query_df)}] 완료: {result.get('translated', '')[:50]}...")
    else:
        # 오류 발생 시 빈 값 추가
        all_results.append(None)
        translated_results.append('')
        keyword_results.append([])
        print(f"[{idx+1}/{len(query_df)}] 오류 발생")
    
    # API 호출 제한을 위한 잠시 대기
    time.sleep(0.5)

print(f"\n번역 완료! 총 {len([r for r in all_results if r is not None])}개 성공")
print(f"오류 발생: {len([r for r in all_results if r is None])}개")


In [ ]:
# 결과 저장
import datetime

# 1. JSONL 파일로 전체 결과 저장
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
jsonl_filename = f"../data/translation_results_{timestamp}.jsonl"

print(f"JSONL 파일 저장: {jsonl_filename}")

with open(jsonl_filename, 'w', encoding='utf-8') as f:
    for result in all_results:
        if result is not None:
            f.write(json.dumps(result, ensure_ascii=False) + '\n')
        else:
            f.write(json.dumps({"error": "translation_failed"}, ensure_ascii=False) + '\n')

print(f"JSONL 파일 저장 완료: {len(all_results)}개 레코드")

# 2. 데이터프레임에 새 컬럼 추가
query_df['translated_4o_mini'] = translated_results
query_df['keyword_4o_mini'] = keyword_results

print("\n데이터프레임 새 컬럼 추가 완료:")
print(f"- translated_4o_mini: {len(translated_results)}개")
print(f"- keyword_4o_mini: {len(keyword_results)}개")

# 3. 업데이트된 CSV 저장
updated_csv_filename = f"../data/helloworld_test_query_with_translation_{timestamp}.csv"
query_df.to_csv(updated_csv_filename, index=False, encoding='utf-8')

print(f"\n업데이트된 CSV 파일 저장: {updated_csv_filename}")


In [ ]:
query_df[['query', 'translated_4o_mini', 'keyword_4o_mini']]
query_df.head()

In [ ]:
for i, row in query_df.iterrows():
    print(row['query'])
    print(row['translated_4o_mini'])
    print(row['keyword_4o_mini'])
    print('-'*100)

## 베이스라인 모델 평가


In [ ]:
import os, sys
import dotenv

# 프로젝트 루트 디렉토리를 Python 경로에 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


dotenv.load_dotenv()

In [ ]:
import pandas as pd
query_df = pd.read_csv("../data/helloworld_test_query_with_translation_20250916_192212.csv")
query_df.head()

In [ ]:
import ast

for i, row in query_df.iterrows():
    print(i, len(ast.literal_eval(row['ground_truth_id'])))

# 최대 20개까지 반환해야 함


In [ ]:
# MongoDB 연결 및 모델 설정
import json
import os
from pymongo import MongoClient
from Azure.model import ChatModel
from dotenv import load_dotenv

# 설정 파일 로드
with open('../configs/config.json', 'r', encoding='utf-8') as f:
    config = json.load(f)


load_dotenv()

# MongoDB 연결
client = MongoClient(os.getenv("MONGODB_URI"))
db = client[config['path']['db_name']]
collection = db[config['path']['collection_name']]

# ChatModel 인스턴스 생성
chat_model = ChatModel(config)

print("MongoDB 연결 및 모델 설정 완료")
print(f"데이터베이스: {config['path']['db_name']}")
print(f"컬렉션: {config['path']['collection_name']}")
print(f"Top-k: {config['chat_config']['top_k']}")

In [ ]:
# Retrieval Correctness 계산 함수
def calculate_retrieval_correctness(retrieved_doc_ids, ground_truth_ids):
    """
    검색된 문서 ID들과 ground truth ID들을 비교하여 correctness 계산
    """
    if not retrieved_doc_ids or not ground_truth_ids:
        return 0
    
    # ground_truth_ids가 문자열 리스트인 경우 처리
    if isinstance(ground_truth_ids, str):
        try:
            # 문자열을 리스트로 변환 (예: "['id1', 'id2']" -> ['id1', 'id2'])
            import ast
            ground_truth_list = ast.literal_eval(ground_truth_ids)
        except:
            ground_truth_list = [ground_truth_ids]
    else:
        ground_truth_list = ground_truth_ids
    
    # 검색된 문서 중 하나라도 ground truth에 있으면 1, 아니면 0
    for doc_id in retrieved_doc_ids:
        if doc_id in ground_truth_list:
            return 1
    
    return 0

print("Retrieval Correctness 계산 함수 정의 완료")

# 확장된 Retrieval 메트릭 계산 함수들
def calculate_recall_at_k(retrieved_doc_ids, ground_truth_ids, k=None):
    """
    Recall@k 계산: 검색된 문서 중 관련 문서의 비율
    """
    if not retrieved_doc_ids or not ground_truth_ids:
        return 0.0
    
    # ground_truth_ids가 문자열 리스트인 경우 처리
    if isinstance(ground_truth_ids, str):
        try:
            import ast
            ground_truth_list = ast.literal_eval(ground_truth_ids)
        except:
            ground_truth_list = [ground_truth_ids]
    else:
        ground_truth_list = ground_truth_ids
    
    # k가 지정되지 않으면 검색된 문서 수만큼 사용
    if k is None:
        k = len(retrieved_doc_ids)
    
    # 상위 k개 문서만 고려
    top_k_retrieved = retrieved_doc_ids[:k]
    
    # 관련 문서 수 계산
    relevant_retrieved = sum(1 for doc_id in top_k_retrieved if doc_id in ground_truth_list)
    
    # Recall = 관련 문서 수 / 전체 관련 문서 수
    if len(ground_truth_list) == 0:
        return 0.0
    
    return relevant_retrieved / len(ground_truth_list)


def calculate_all_metrics(retrieved_doc_ids, ground_truth_ids, k=None):
    """
    모든 메트릭을 한 번에 계산
    """
    return {
        "correctness": calculate_retrieval_correctness(retrieved_doc_ids, ground_truth_ids),
        "recall_at_k": calculate_recall_at_k(retrieved_doc_ids, ground_truth_ids, k)
    }

print("확장된 Retrieval 메트릭 계산 함수들 정의 완료")


In [ ]:
# 수정된 모델 답변 생성 및 문서 인덱스 추출 함수 (중복 검색 제거)
def get_model_response_with_docs(query_text):
    """
    모델로부터 답변을 생성하고 검색된 문서들의 인덱스를 반환
    이제 model.py의 generate_ai_response가 검색된 문서 ID도 함께 반환하므로 중복 검색 제거
    """
    try:
        # 빈 대화 히스토리로 시작
        conversation_history = []
        
        # 모델 답변 생성 (이제 검색된 문서 ID도 함께 반환됨)
        response = chat_model.generate_ai_response(conversation_history, query_text, collection)
        
        # model.py에서 이미 반환된 정보를 그대로 사용
        return {
            "answer": response["answer"],
            "retrieved_doc_ids": response["retrieved_doc_ids"],
            "retrieved_docs": response["retrieved_docs"]
        }
        
    except Exception as e:
        print(f"오류 발생: {e}")
        return {
            "answer": "",
            "retrieved_doc_ids": [],
            "retrieved_docs": []
        }

print("수정된 모델 응답 및 문서 인덱스 추출 함수 정의 완료 (중복 검색 제거됨)")


In [ ]:
# 확장된 메트릭을 사용한 평가 실행
import time
from tqdm import tqdm

# 결과를 저장할 리스트들
baseline_results = []
retrieved_doc_ids_list = []

# 메트릭별 결과 저장
correctness_scores = []
recall_at_k_scores = []
precision_at_k_scores = []
f1_at_k_scores = []

print(f"총 {len(query_df)}개의 쿼리에 대해 확장된 메트릭 평가를 시작합니다...")

# 각 쿼리에 대해 평가 실행
for idx, row in tqdm(query_df.iterrows(), total=len(query_df), desc="확장된 메트릭 평가"):
    query_text = row['translated_4o_mini']
    ground_truth_ids = row['ground_truth_id']
    
    print(f"\n[{idx+1}/{len(query_df)}] 처리 중: {query_text[:50]}...")
    
    # 수정된 함수 사용 (중복 검색 제거됨)
    result = get_model_response_with_docs(query_text)
    
    # 결과 저장
    baseline_results.append(result['answer'])
    retrieved_doc_ids_list.append(result['retrieved_doc_ids'])
    
    # 모든 메트릭 계산
    metrics = calculate_all_metrics(result['retrieved_doc_ids'], ground_truth_ids)
    
    # 메트릭별 점수 저장
    correctness_scores.append(metrics['correctness'])
    recall_at_k_scores.append(metrics['recall_at_k'])
    
    print(f"검색된 문서 수: {len(result['retrieved_doc_ids'])}")
    print(f"Correctness: {metrics['correctness']}")
    print(f"Recall@k: {metrics['recall_at_k']:.3f}")
    
    # API 호출 제한을 위한 잠시 대기
    time.sleep(1)

# 전체 결과 요약
print(f"\n=== 확장된 메트릭 평가 완료! ===")
print(f"평균 Correctness: {sum(correctness_scores) / len(correctness_scores):.3f}")
print(f"평균 Recall@k: {sum(recall_at_k_scores) / len(recall_at_k_scores):.3f}")


# 결과를 데이터프레임에 저장 및 CSV 내보내기
import datetime

# 데이터프레임에 새 컬럼들 추가
query_df['answer_baseline'] = baseline_results
query_df['retrieved_doc_ids'] = retrieved_doc_ids_list
query_df['correctness'] = correctness_scores
query_df['recall_at_k'] = recall_at_k_scores

# 결과 저장
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
extended_metrics_csv_filename = f"../data/extended_metrics_evaluation_results_{timestamp}.csv"
query_df.to_csv(extended_metrics_csv_filename, index=False, encoding='utf-8')

print(f"\n결과 저장 완료:")
print(f"- answer_baseline: {len(baseline_results)}개")
print(f"- retrieved_doc_ids: {len(retrieved_doc_ids_list)}개") 
print(f"- correctness: {len(correctness_scores)}개")
print(f"- recall_at_k: {len(recall_at_k_scores)}개")
print(f"- CSV 파일: {extended_metrics_csv_filename}")

# 평가 결과 상세 요약
total_queries = len(query_df)
correct_retrievals = sum(correctness_scores)
avg_correctness = correct_retrievals / total_queries
avg_recall = sum(recall_at_k_scores) / len(recall_at_k_scores)

print(f"\n=== 확장된 메트릭 평가 결과 요약 ===")
print(f"총 쿼리 수: {total_queries}")
print(f"정확한 검색 수: {correct_retrievals}")
print(f"평균 Correctness: {avg_correctness:.3f} ({avg_correctness*100:.1f}%)")
print(f"평균 Recall@k: {avg_recall:.3f} ({avg_recall*100:.1f}%)")



In [ ]:
df = pd.read_csv("../data/extended_metrics_evaluation_results_20250916_203357.csv")
df.head()

# for i, row in df.iterrows():
#     print(len(ast.literal_eval(row["retrieved_doc_ids"])))

### 새로운 검색 성능

In [8]:
import os, sys
import pandas as pd

# 프로젝트 루트 디렉토리를 Python 경로에 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

query_df = pd.read_csv("../data/helloworld_test_query_with_translation_20250916_192212.csv")
query_df.head()

,query,translated_query,언어,ground_truth_id,category,source,작성자,비고,소스,translated_4o_mini,keyword_4o_mini
0,"제 여자친구가 단속으로 출입국 보호소에 있습니다. 월급이 아직 들어오지 않았는데, ...",แฟนของผมถูกจับกุมและอยู่ที่ศูนย์กักตัวตรวจคนเข...,태국어,"['689b3a86ffd306c1cd3c09a4', '689b3a86ffd306c1...",임금체불,경기도외국인지원센터_상담사례,황예원,NaN,https://gmhr.or.kr/case/1529?sca=%EC%9E%84%EA%...,제 여자친구가 체포되어 출입국 관리소에 구금되어 있습니다. 지금 월급을 받지 못했는...,"['체포', '출입국 관리소', '월급', '밀린 월급']"
1,"안녕하세요, 건설 현장에서 일하고 있는 사람인데, 사장님이 월급을 안줘서 계좌가 압...",你好，我是在建筑工地工作的，但老板没有发工资，我的账户可能会被查封。遇到这种情况该怎么办呢？,중국어,"['689b3a86ffd306c1cd3c06e8', '689b3a86ffd306c1...",임금체불,경기도외국인지원센터_상담사례,황예원,"임금체불 및 ""압류방지 통장"" (=임금채권 전용통장) 관련 데이터 필요",https://gmhr.or.kr/case/1493?sca=%EC%9E%84%EA%...,"안녕하세요, 저는 건설 현장에서 일하고 있는데, 사장이 급여를 지급하지 않았습니다....","['건설업', '임금체불', '계좌 압류']"
2,"안녕하세요, 저는 필리핀에서 온 노동자입니다. 5년 동안 근무를 하고 이제 제 나라...","Magandang araw, ako ay isang manggagawang mula...",필리핀어 (타갈로그어),['689b3a86ffd306c1cd3c09a4'],임금체불,경기도외국인지원센터_상담사례,황예원,체당금 관련 데이터 필요,https://gmhr.or.kr/case/1667?sca=%EC%9E%84%EA%...,"안녕하세요, 저는 필리핀에서 온 노동자입니다. 5년 동안 일했는데 이제 고국으로 돌...","['임금체불', '회수 방법']"
3,사업장 변경 신청 이후 제가 불법체류자가 될 수 있다는 우편이 날아왔어요. 8월 2...,在申请变更工作单位之后，我收到了一封信，说我可能会变成非法滞留者。只被允许停留到8月22日，...,중국어,"['689b3a86ffd306c1cd3c0680', '689b3a86ffd306c1...",체류자격,경기도외국인지원센터_상담사례,황예원,NaN,https://gmhr.or.kr/case/1679?sca=%EC%B2%B4%EB%...,"사업장 변경을 신청한 후, 제가 불법 체류자가 될 수 있다는 내용의 편지를 받았습니...","['사업장 변경', '불법 체류', '추방', '체류 허용 기간']"
4,"제가 중간에 퇴직을 하게 되었는데, 소득세가 체납되어 비자 연장이 안된대요. 그런데...",我中途离职了，但是因为拖欠所得税，签证无法延期。可是我听不懂相关的通知内容。,중국어,"['689b3a86ffd306c1cd3c08f8', '689b3a86ffd306c1...",체류자격,경기도외국인지원센터_상담사례,황예원,NaN,https://gmhr.or.kr/case/1703?sca=%EC%B2%B4%EB%...,"저는 중간에 퇴사했지만, 소득세 체납 때문에 비자를 연장할 수 없습니다. 그런데 관...","['퇴사', '소득세 체납', '비자 연장', '통지 내용']"


In [9]:
# 키워드 기반 하이브리드 검색 모델 평가
from Azure.keyword_model import ChatModel as KeywordChatModel
from dotenv import load_dotenv
import json

# 환경 설정
load_dotenv()

# 설정 파일 로드
with open('../configs/config.json', 'r', encoding='utf-8') as f:
    config = json.load(f)

# 키워드 모델 인스턴스 생성
keyword_chat_model = KeywordChatModel(config)

print("키워드 기반 하이브리드 검색 모델 설정 완료")

# 키워드 기반 검색 함수
def get_keyword_model_response_with_docs(query_text, keywords):
    """
    키워드 기반 하이브리드 검색 모델로부터 답변을 생성하고 검색된 문서들의 인덱스를 반환
    """
    try:
        # 빈 대화 히스토리로 시작
        conversation_history = []
        
        # 키워드 기반 모델 답변 생성
        response = keyword_chat_model.generate_ai_response(
            conversation_history, 
            query_text, 
            collection, 
            keywords=keywords
        )
        
        return {
            "answer": response["answer"],
            "retrieved_doc_ids": response["retrieved_doc_ids"],
            "retrieved_docs": response["retrieved_docs"]
        }
        
    except Exception as e:
        print(f"오류 발생: {e}")
        return {
            "answer": "",
            "retrieved_doc_ids": [],
            "retrieved_docs": []
        }

print("키워드 기반 검색 함수 정의 완료")


키워드 기반 하이브리드 검색 모델 설정 완료
키워드 기반 검색 함수 정의 완료


In [11]:
# Retrieval Correctness 계산 함수
def calculate_retrieval_correctness(retrieved_doc_ids, ground_truth_ids):
    """
    검색된 문서 ID들과 ground truth ID들을 비교하여 correctness 계산
    """
    if not retrieved_doc_ids or not ground_truth_ids:
        return 0
    
    # ground_truth_ids가 문자열 리스트인 경우 처리
    if isinstance(ground_truth_ids, str):
        try:
            # 문자열을 리스트로 변환 (예: "['id1', 'id2']" -> ['id1', 'id2'])
            import ast
            ground_truth_list = ast.literal_eval(ground_truth_ids)
        except:
            ground_truth_list = [ground_truth_ids]
    else:
        ground_truth_list = ground_truth_ids
    
    # 검색된 문서 중 하나라도 ground truth에 있으면 1, 아니면 0
    for doc_id in retrieved_doc_ids:
        if doc_id in ground_truth_list:
            return 1
    
    return 0

print("Retrieval Correctness 계산 함수 정의 완료")

# 확장된 Retrieval 메트릭 계산 함수들
def calculate_recall_at_k(retrieved_doc_ids, ground_truth_ids, k=None):
    """
    Recall@k 계산: 검색된 문서 중 관련 문서의 비율
    """
    if not retrieved_doc_ids or not ground_truth_ids:
        return 0.0
    
    # ground_truth_ids가 문자열 리스트인 경우 처리
    if isinstance(ground_truth_ids, str):
        try:
            import ast
            ground_truth_list = ast.literal_eval(ground_truth_ids)
        except:
            ground_truth_list = [ground_truth_ids]
    else:
        ground_truth_list = ground_truth_ids
    
    # k가 지정되지 않으면 검색된 문서 수만큼 사용
    if k is None:
        k = len(retrieved_doc_ids)
    
    # 상위 k개 문서만 고려
    top_k_retrieved = retrieved_doc_ids[:k]
    
    # 관련 문서 수 계산
    relevant_retrieved = sum(1 for doc_id in top_k_retrieved if doc_id in ground_truth_list)
    
    # Recall = 관련 문서 수 / 전체 관련 문서 수
    if len(ground_truth_list) == 0:
        return 0.0
    
    return relevant_retrieved / len(ground_truth_list)


def calculate_all_metrics(retrieved_doc_ids, ground_truth_ids, k=None):
    """
    모든 메트릭을 한 번에 계산
    """
    return {
        "correctness": calculate_retrieval_correctness(retrieved_doc_ids, ground_truth_ids),
        "recall_at_k": calculate_recall_at_k(retrieved_doc_ids, ground_truth_ids, k)
    }

print("확장된 Retrieval 메트릭 계산 함수들 정의 완료")


Retrieval Correctness 계산 함수 정의 완료
확장된 Retrieval 메트릭 계산 함수들 정의 완료


In [13]:
# MongoDB 연결 테스트
import json
import os
import pymongo
from pymongo import MongoClient

print("=" * 50)
print("MongoDB 연결 테스트 시작")
print("=" * 50)

# 설정 파일 로드
with open('../configs/config.json', 'r', encoding='utf-8') as f:
    config = json.load(f)

# MongoDB 클라이언트 연결
mongodb_client = MongoClient(os.getenv("MONGODB_URI"))
print("🔗 MongoDB 클라이언트 연결 완료")

try:
    # 연결 상태 확인
    mongodb_client.admin.command('ping')
    print("✅ MongoDB 연결 성공!")
    
    # 서버 정보 가져오기
    server_info = mongodb_client.server_info()
    print(f"📊 MongoDB 버전: {server_info['version']}")
    
    # 데이터베이스 목록 확인
    db_list = mongodb_client.list_database_names()
    print(f"📁 사용 가능한 데이터베이스: {db_list}")
    
    # 현재 데이터베이스 정보
    current_db = mongodb_client[config['path']['db_name']]
    print(f"🎯 현재 데이터베이스: {config['path']['db_name']}")
    
    # 컬렉션 목록 확인
    collections = current_db.list_collection_names()
    print(f"📋 컬렉션 목록: {collections}")
    
    # 타겟 컬렉션 확인
    target_collection = current_db[config['path']['collection_name']]
    print(f"🎯 타겟 컬렉션: {config['path']['collection_name']}")
    
    # 컬렉션 통계 정보
    stats = current_db.command("collStats", config['path']['collection_name'])
    print(f"📈 문서 개수: {stats['count']:,}")
    print(f"💾 컬렉션 크기: {stats['size']:,} bytes ({stats['size']/1024/1024:.2f} MB)")
    
    # 샘플 문서 확인
    sample_doc = target_collection.find_one()
    if sample_doc:
        print(f"📄 샘플 문서 키: {list(sample_doc.keys())}")
        print(f"📄 샘플 문서 ID: {sample_doc.get('_id', 'N/A')}")
    else:
        print("⚠️ 컬렉션에 문서가 없습니다.")
    
    # 인덱스 정보 확인
    indexes = target_collection.list_indexes()
    print(f"🔍 인덱스 정보:")
    for idx in indexes:
        print(f"   - {idx['name']}: {idx['key']}")
    
    print("=" * 50)
    print("✅ MongoDB 연결 테스트 완료!")
    print("=" * 50)
    
except pymongo.errors.ConnectionFailure as e:
    print(f"❌ MongoDB 연결 실패: {e}")
except pymongo.errors.ServerSelectionTimeoutError as e:
    print(f"❌ 서버 선택 타임아웃: {e}")
except Exception as e:
    print(f"❌ 예상치 못한 오류: {e}")


MongoDB 연결 테스트 시작
🔗 MongoDB 클라이언트 연결 완료
✅ MongoDB 연결 성공!
📊 MongoDB 버전: 8.0.13
📁 사용 가능한 데이터베이스: ['HelloWorld-AI', 'admin', 'local']
🎯 현재 데이터베이스: HelloWorld-AI
📋 컬렉션 목록: ['foreigner_legalQA_v2', 'foreigner_legal_test', 'foreigner_legalQA', 'foreigner_legalQA_v3']
🎯 타겟 컬렉션: foreigner_legalQA_v3
📈 문서 개수: 867
💾 컬렉션 크기: 37,616,750 bytes (35.87 MB)
📄 샘플 문서 키: ['_id', 'title', 'contents', 'url', 'Embedding']
📄 샘플 문서 ID: 689b3a86ffd306c1cd3c0679
🔍 인덱스 정보:
   - _id_: SON([('_id', 1)])
✅ MongoDB 연결 테스트 완료!


In [15]:
# MongoDB 연결 및 모델 설정
import json
import os
from pymongo import MongoClient
from Azure.model import ChatModel
from dotenv import load_dotenv

# 설정 파일 로드
with open('../configs/config.json', 'r', encoding='utf-8') as f:
    config = json.load(f)


load_dotenv()

# MongoDB 연결
client = MongoClient(os.getenv("MONGODB_URI"))
db = client[config['path']['db_name']]
collection = db[config['path']['collection_name']]

# ChatModel 인스턴스 생성
chat_model = ChatModel(config)

print("MongoDB 연결 및 모델 설정 완료")
print(f"데이터베이스: {config['path']['db_name']}")
print(f"컬렉션: {config['path']['collection_name']}")
print(f"Top-k: {config['chat_config']['top_k']}")

MongoDB 연결 및 모델 설정 완료
데이터베이스: HelloWorld-AI
컬렉션: foreigner_legalQA_v3
Top-k: 20


In [17]:
query_df.columns

Index(['query', 'translated_query', '언어', 'ground_truth_id', 'category',
       'source', '작성자', '비고', '소스', 'translated_4o_mini', 'keyword_4o_mini'],
      dtype='object')

In [19]:
# 키워드 기반 하이브리드 검색 평가 실행
import time
from tqdm import tqdm
import ast

# 결과를 저장할 리스트들
keyword_baseline_results = []
keyword_retrieved_doc_ids_list = []

# 메트릭별 결과 저장
keyword_correctness_scores = []
keyword_recall_at_k_scores = []

print(f"총 {len(query_df)}개의 쿼리에 대해 키워드 기반 하이브리드 검색 평가를 시작합니다...")

# 각 쿼리에 대해 평가 실행
for idx, row in tqdm(query_df.iterrows(), total=len(query_df), desc="키워드 하이브리드 검색 평가"):
    query_text = row['translated_4o_mini']
    ground_truth_ids = row['ground_truth_id']
    
    # 키워드 추출 및 처리
    try:
        keywords_raw = row['keyword_4o_mini']
        if isinstance(keywords_raw, str):
            keywords = ast.literal_eval(keywords_raw)
        else:
            keywords = keywords_raw
    except Exception as e:
        raise e
    
    print(f"\n[{idx+1}/{len(query_df)}] 처리 중: {query_text[:50]}...")
    print(f"사용할 키워드: {keywords}")
    
    # 키워드 기반 모델 사용
    result = get_keyword_model_response_with_docs(query_text, keywords)
    
    # 결과 저장
    keyword_baseline_results.append(result['answer'])
    keyword_retrieved_doc_ids_list.append(result['retrieved_doc_ids'])
    
    # 모든 메트릭 계산
    metrics = calculate_all_metrics(result['retrieved_doc_ids'], ground_truth_ids)
    
    # 메트릭별 점수 저장
    keyword_correctness_scores.append(metrics['correctness'])
    keyword_recall_at_k_scores.append(metrics['recall_at_k'])
    
    print(f"검색된 문서 수: {len(result['retrieved_doc_ids'])}")
    print(f"Correctness: {metrics['correctness']}")
    print(f"Recall@k: {metrics['recall_at_k']:.3f}")
    
    # API 호출 제한을 위한 잠시 대기
    time.sleep(1)

# 전체 결과 요약
print(f"\n=== 키워드 기반 하이브리드 검색 평가 완료! ===")
print(f"평균 Correctness: {sum(keyword_correctness_scores) / len(keyword_correctness_scores):.3f}")
print(f"평균 Recall@k: {sum(keyword_recall_at_k_scores) / len(keyword_recall_at_k_scores):.3f}")


총 20개의 쿼리에 대해 키워드 기반 하이브리드 검색 평가를 시작합니다...


키워드 하이브리드 검색 평가:   0%|          | 0/20 [00:00<?, ?it/s]


[1/20] 처리 중: 제 여자친구가 체포되어 출입국 관리소에 구금되어 있습니다. 지금 월급을 받지 못했는데, 만...
사용할 키워드: ['체포', '출입국 관리소', '월급', '밀린 월급']
검색된 문서 수: 20
Correctness: 1
Recall@k: 0.214


키워드 하이브리드 검색 평가:   5%|▌         | 1/20 [00:13<04:13, 13.32s/it]


[2/20] 처리 중: 안녕하세요, 저는 건설 현장에서 일하고 있는데, 사장이 급여를 지급하지 않았습니다. 제 계...
사용할 키워드: ['건설업', '임금체불', '계좌 압류']
검색된 문서 수: 20
Correctness: 1
Recall@k: 0.375


...


[19/20] 처리 중: 해외 만기 보험금을 신청하려면 어떤 자격 조건을 충족해야 하나요? 언제부터 신청할 수 있나...
사용할 키워드: ['해외 만기 보험금', '자격 조건', '신청 시기']
검색된 문서 수: 20
Correctness: 0
Recall@k: 0.000


키워드 하이브리드 검색 평가:  95%|█████████▌| 19/20 [03:20<00:10, 10.12s/it]


[20/20] 처리 중: 고국으로 돌아갈 때, 언제부터 귀국비용 보험을 신청할 수 있으며, 어떤 서류를 준비해야 하...
사용할 키워드: ['귀국비용 보험', '신청 시기', '필요 서류']
검색된 문서 수: 20
Correctness: 0
Recall@k: 0.000


키워드 하이브리드 검색 평가: 100%|██████████| 20/20 [03:28<00:00, 10.40s/it]


=== 키워드 기반 하이브리드 검색 평가 완료! ===
평균 Correctness: 0.600
평균 Recall@k: 0.289


In [22]:
# 키워드 기반 검색 결과 저장 및 비교 분석
import datetime

# 데이터프레임에 키워드 기반 검색 결과 추가
query_df['answer_keyword_hybrid'] = keyword_baseline_results
query_df['retrieved_doc_ids_keyword'] = keyword_retrieved_doc_ids_list
query_df['correctness_keyword'] = keyword_correctness_scores
query_df['recall_at_k_keyword'] = keyword_recall_at_k_scores

# 결과 저장
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
keyword_evaluation_csv_filename = f"../data/keyword_hybrid_evaluation_results_{timestamp}.csv"
query_df.to_csv(keyword_evaluation_csv_filename, index=False, encoding='utf-8')

print(f"\n키워드 기반 검색 결과 저장 완료:")
print(f"- answer_keyword_hybrid: {len(keyword_baseline_results)}개")
print(f"- retrieved_doc_ids_keyword: {len(keyword_retrieved_doc_ids_list)}개") 
print(f"- correctness_keyword: {len(keyword_correctness_scores)}개")
print(f"- recall_at_k_keyword: {len(keyword_recall_at_k_scores)}개")
print(f"- CSV 파일: {keyword_evaluation_csv_filename}")

# 평가 결과 상세 요약
total_queries = len(query_df)
keyword_correct_retrievals = sum(keyword_correctness_scores)
keyword_avg_correctness = keyword_correct_retrievals / total_queries
keyword_avg_recall = sum(keyword_recall_at_k_scores) / len(keyword_recall_at_k_scores)

print(f"\n=== 키워드 기반 하이브리드 검색 평가 결과 요약 ===")
print(f"총 쿼리 수: {total_queries}")
print(f"정확한 검색 수: {keyword_correct_retrievals}")
print(f"평균 Correctness: {keyword_avg_correctness:.3f} ({keyword_avg_correctness*100:.1f}%)")
print(f"평균 Recall@k: {keyword_avg_recall:.3f} ({keyword_avg_recall*100:.1f}%)")



키워드 기반 검색 결과 저장 완료:
- answer_keyword_hybrid: 20개
- retrieved_doc_ids_keyword: 20개
- correctness_keyword: 20개
- recall_at_k_keyword: 20개
- CSV 파일: ../data/keyword_hybrid_evaluation_results_20250916_210503.csv

=== 키워드 기반 하이브리드 검색 평가 결과 요약 ===
총 쿼리 수: 20
정확한 검색 수: 12
평균 Correctness: 0.600 (60.0%)
평균 Recall@k: 0.289 (28.9%)


In [25]:
df = pd.read_csv("../data/keyword_hybrid_evaluation_results_20250916_205256.csv")

In [28]:
df.columns

Index(['query', 'translated_query', '언어', 'ground_truth_id', 'category',
       'source', '작성자', '비고', '소스', 'translated_4o_mini', 'keyword_4o_mini',
       'answer_baseline', 'retrieved_doc_ids', 'correctness', 'recall_at_k',
       'answer_keyword_hybrid', 'retrieved_doc_ids_keyword',
       'correctness_keyword', 'recall_at_k_keyword'],
      dtype='object')

In [27]:
df['correctness_keyword'].mean()

np.float64(0.6)

In [29]:
df['recall_at_k_keyword'].mean()

np.float64(0.2894642857142857)

In [30]:
df2 = pd.read_csv("../data/keyword_hybrid_evaluation_results_20250916_210409.csv")

In [31]:
df2['correctness_keyword'].mean()


np.float64(0.6)

In [32]:
df2['recall_at_k_keyword'].mean()

np.float64(0.2894642857142857)

In [23]:
# 기존 벡터 검색 vs 키워드 기반 하이브리드 검색 성능 비교
print("=" * 80)
print("🔍 검색 성능 비교 분석")
print("=" * 80)

# 기존 벡터 검색 결과 (이전 평가에서)
baseline_avg_correctness = sum(correctness_scores) / len(correctness_scores)
baseline_avg_recall = sum(recall_at_k_scores) / len(recall_at_k_scores)

print(f"\n📊 성능 비교:")
print(f"{'메트릭':<20} {'벡터 검색':<15} {'키워드 하이브리드':<20} {'개선도':<15}")
print("-" * 70)
print(f"{'Correctness':<20} {baseline_avg_correctness:.3f} ({baseline_avg_correctness*100:.1f}%){'':<5} {keyword_avg_correctness:.3f} ({keyword_avg_correctness*100:.1f}%){'':<5} {((keyword_avg_correctness - baseline_avg_correctness) / baseline_avg_correctness * 100):+.1f}%")
print(f"{'Recall@k':<20} {baseline_avg_recall:.3f} ({baseline_avg_recall*100:.1f}%){'':<5} {keyword_avg_recall:.3f} ({keyword_avg_recall*100:.1f}%){'':<5} {((keyword_avg_recall - baseline_avg_recall) / baseline_avg_recall * 100):+.1f}%")

# 개선된 쿼리 수 계산
correctness_improved = sum(1 for i in range(len(query_df)) if keyword_correctness_scores[i] > correctness_scores[i])
recall_improved = sum(1 for i in range(len(query_df)) if keyword_recall_at_k_scores[i] > recall_at_k_scores[i])

print(f"\n📈 개선 통계:")
print(f"Correctness 개선된 쿼리: {correctness_improved}/{len(query_df)}개 ({correctness_improved/len(query_df)*100:.1f}%)")
print(f"Recall@k 개선된 쿼리: {recall_improved}/{len(query_df)}개 ({recall_improved/len(query_df)*100:.1f}%)")

# 상세 비교를 위한 샘플 출력
print(f"\n📋 상세 비교 샘플 (상위 5개):")
comparison_df = query_df[['query', 'correctness', 'recall_at_k', 'correctness_keyword', 'recall_at_k_keyword']].head(5)

for idx, row in comparison_df.iterrows():
    print(f"\n[샘플 {idx+1}]")
    print(f"쿼리: {row['query'][:60]}...")
    print(f"벡터 검색 - Correctness: {row['correctness']}, Recall@k: {row['recall_at_k']:.3f}")
    print(f"키워드 하이브리드 - Correctness: {row['correctness_keyword']}, Recall@k: {row['recall_at_k_keyword']:.3f}")

print("\n" + "=" * 80)
print("✅ 키워드 기반 하이브리드 검색 평가 완료!")
print("=" * 80)


🔍 검색 성능 비교 분석


NameError: name 'correctness_scores' is not defined